<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

ResNet50 = ResNet50(weights='imagenet')
features = model.predict(x)

```

Next you will need to remove the last layer from the ResNet model. Here, we loop over the layers to use the sequential API. There are easier ways to add and remove layers using the Keras functional API, but doing so introduces other complexities. 

```python
# Remote the Last Layer of ResNEt
ResNet50._layers.pop(0)

# Out New Model
model = Sequential()

# Add Pre-trained layers of Old Model to New Model
for layer in ResNet50.layers:
    model.add(layer)

# Turn off additional training of ResNet Layers for speed of assignment
for layer in model.layers:
    layer.trainable = False

# Add New Output Layer to Model
model.add(Dense(1, activation='sigmoid'))
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

In [0]:
### YOUR CODE HERE
import cv2
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
# from skimage import io
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D

In [0]:
!wget https://github.com/1aaronscott/DS-Unit-4-Sprint-3-Deep-Learning/blob/mychanges/module2-convolutional-neural-networks/data.tgz?raw=true -O data.tgz

In [0]:
!tar xzf data.tgz

In [43]:
DATADIR = 'data'
CATEGORIES = ['forest', 'mountain']
training_data = []
IMG_SIZE = 224

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category) # path to cats or dogs dir
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                if not img.startswith(".jpg"):
                    img_array = cv2.imread(os.path.join(path, img))
                    new_img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                    training_data.append([new_img_array, class_num])
            except Exception as e:
                pass
    X = []
    y = []

    for features, label in training_data:
        X.append(features)
        y.append(label)

    X = np.array(X)
    y = to_categorical(np.array(y))

    print('------- Shape of X, y data -------')
    print(X.shape, y.shape)

    return X/255.0, y #normalize the data
X, y = create_training_data()

------- Shape of X, y data -------
(702, 224, 224, 3) (702, 2)


In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((561, 224, 224, 3), (141, 224, 224, 3), (561, 2), (141, 2))

In [0]:
fig = plt.figure(figsize=(15, 10))
columns = 3
rows = 2
for i in range(1, columns*rows +1):
    ax = fig.add_subplot(rows, columns, i)
    plt.imshow((X_train[i]* 255).astype(np.uint8))
    ax.set_xlabel(y[np.argmax(y_train[i])])
plt.show();



In [0]:
%%time
res = ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

# make all resnet layers untrainable
for layer in res.layers:
    layer.trainable = False
    
# add your head on top
x = res.output
x = MaxPooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=res.input, outputs=predictions)

# x = res.output
# predictions = Dense(1, activation='sigmoid')(x)
# model = Model(res.input, predictions)
model.summary()

In [0]:
%%time
res = ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

# make all resnet layers untrainable
for layer in res.layers:
    layer.trainable = False
    
# add your head on top
x = res.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=res.input, outputs=predictions)

# x = res.output
# predictions = Dense(1, activation='sigmoid')(x)
# model = Model(res.input, predictions)
model.summary()

In [48]:
%%time
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

CPU times: user 106 ms, sys: 1.01 ms, total: 107 ms
Wall time: 109 ms


In [49]:
%%time

EPOCHS = 5
BATCH_SIZE = 10

history = model.fit(X_train, y_train, epochs=EPOCHS, validation_split=0.2, batch_size=BATCH_SIZE, verbose=1)

Train on 448 samples, validate on 113 samples
Epoch 1/5
448/448 [==============================] - 148s 331ms/sample - loss: 0.5017 - acc: 0.8638 - val_loss: 0.6945 - val_acc: 0.4779
Epoch 2/5
448/448 [==============================] - 141s 315ms/sample - loss: 0.3199 - acc: 0.9241 - val_loss: 1.4465 - val_acc: 0.4956
Epoch 3/5
448/448 [==============================] - 142s 316ms/sample - loss: 0.2745 - acc: 0.9442 - val_loss: 0.9631 - val_acc: 0.4956
Epoch 4/5
448/448 [==============================] - 142s 316ms/sample - loss: 0.1320 - acc: 0.9531 - val_loss: 0.9444 - val_acc: 0.4956
Epoch 5/5
448/448 [==============================] - 142s 318ms/sample - loss: 0.0744 - acc: 0.9732 - val_loss: 1.1222 - val_acc: 0.4956
CPU times: user 23min 11s, sys: 15.8 s, total: 23min 27s
Wall time: 12min 3s


In [32]:
X_train[0].shape

(224, 224, 3)

In [51]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

141/141 - 29s - loss: 0.9282 - acc: 0.5887


# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language